In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# Read the geopandas dataframe, from a parquet, containing the synthetic trajectories.
gdf = gpd.read_parquet('./trajs.parquet')
gdf

,timestamp,geometry,ID
0,2019-07-01 00:00:00,POINT (-84.40031 33.75035),0
1,2019-07-01 00:00:00,POINT (-84.3743 33.73444),1
2,2019-07-01 00:00:00,POINT (-84.37928 33.734),2
3,2019-07-01 00:00:00,POINT (-84.36595 33.73923),3
4,2019-07-01 00:00:00,POINT (-84.38093 33.75587),4
...,...,...,...
1196279,2019-07-03 00:01:00,POINT (-84.41069 33.75307),995
1196280,2019-07-03 00:01:00,POINT (-84.37174 33.74393),996
1196281,2019-07-03 00:01:00,POINT (-84.3659 33.75888),997
1196282,2019-07-03 00:01:00,POINT (-84.41085 33.75312),998


In [3]:
# Randomly select a certain number of users from the dataset. 
n_users = 5
list_users = np.random.choice(gdf['ID'].nunique(), n_users, replace=False).tolist()

users = gdf.loc[gdf['ID'].isin(list_users), :]
print(users.head())

     timestamp                    geometry   ID
241 2019-07-01   POINT (-84.3697 33.75963)  241
339 2019-07-01  POINT (-84.36973 33.75312)  339
707 2019-07-01  POINT (-84.37478 33.73401)  707
727 2019-07-01  POINT (-84.36566 33.74193)  727
799 2019-07-01  POINT (-84.39745 33.73665)  799


In [4]:
# Plot how the users move over time.

import folium
from folium.plugins import TimestampedGeoJson


# 2. Pick a palette of colors and map each unique ID to one
palette = [
    "red", "blue", "green", "purple", "orange",
    "darkred", "lightred", "beige", "darkblue",
    "darkgreen", "cadetblue", "darkpurple"
]
unique_ids = users["ID"].unique()
color_map = {
    uid: palette[i % len(palette)]
    for i, uid in enumerate(unique_ids)
}


# 2. Build a GeoJSON-style dict with times
features = []
for _, row in users.iterrows():
    cid = row["ID"]
    features.append({
        "type": "Feature",
        "geometry": row.geometry.__geo_interface__,
        "properties": {
            "time": row.timestamp.isoformat(),
            "icon": "circle",
            "iconstyle": {
                "fillColor": color_map[cid],
                "fillOpacity": 0.7,
                "stroke": False,
                "radius": 10
            },
            "popup": f"ID {cid}<br>{row.timestamp:%Y-%m-%d %H:%M}"
        }
    })

geojson = {"type": "FeatureCollection", "features": features}


# 3. Create the map
m = folium.Map(location=[users.geometry.y.mean(), users.geometry.x.mean()], zoom_start=13, control_scale=True)

# 4. Add the time‐slider layer
duration_step_minutes = 1
TimestampedGeoJson(
    data=geojson,
    transition_time=200,      # milliseconds between frames
    period=f"PT{duration_step_minutes}M",            # matches your n-minutes stepping
    duration=f"PT{duration_step_minutes}M",
    add_last_point=False,
    auto_play=False,
    loop=False,
).add_to(m)


# 5. Display or save
# m
m.save("trajectory_time_slider.html")